In [ ]:
# Packages
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

In [ ]:
def get_news_urls(links_site):
    '''scrape the html of the site'''
    resp = requests.get(links_site)
 
    if not resp.ok:
        return None
 
    html = resp.content
 
    '''convert html to BeautifulSoup object'''
    soup = BeautifulSoup(html , 'lxml')
 
    '''get list of all links on webpage'''
    links = soup.find_all('a')
 
    urls = [link.get('href') for link in links]
    urls = [url for url in urls if url is not None]
 
    '''Filter the list of urls to just the news articles'''
    news_urls = [url for url in urls if '/article/' in url]
 
    return news_urls

In [ ]:
def scrape_news_text(news_url):
 
    news_html = requests.get(news_url).content
 
    '''convert html to BeautifulSoup object'''
    news_soup = BeautifulSoup(news_html , 'lxml')
 
    paragraphs = [par.text for par in news_soup.find_all('p')]
    news_text = '\n'.join(paragraphs)
    print("Finished scrapping: ", news_url)
 
    return news_text

In [ ]:
def scrape_all_articles(ticker , upper_page_limit = 5):
 
    landing_site = 'http://www.nasdaq.com/symbol/' + ticker + '/news-headlines'
 
    all_news_urls = get_news_urls(landing_site)
 
    current_urls_list = all_news_urls.copy()
 
    index = 2
 
    '''Loop through each sequential page, scraping the links from each'''
    while (current_urls_list is not None) and (current_urls_list != []) and \
        (index <= upper_page_limit):
 
        '''Construct URL for page in loop based off index'''
        current_site = landing_site + '?page=' + str(index)
        current_urls_list = get_news_urls(current_site)
 
        '''Append current webpage's list of urls to all_news_urls'''
        all_news_urls = all_news_urls + current_urls_list
 
        index = index + 1
        print("Done processing page: ", index)
 
    all_news_urls = list(set(all_news_urls))
 
    '''Now, we have a list of urls, we need to actually scrape the text'''
    all_articles = [scrape_news_text(news_url) for news_url in all_news_urls]
 
    return all_articles

In [ ]:
def save_articles_to_files(articles, ticker):
    article_counter = 0
    for article in articles: 
        article_file = ticker + str(article_counter) + '.txt'
        file_name = os.path.join('article_data', ticker, article_file)

        f = open(file_name, 'w')
        f.write(article)
        f.close()
        article_counter += 1
        print("Processed Article Number: ", article_counter)


In [ ]:
def get_processed_articles(articles):
    ad_string = "Enter up to 25 symbols separated by commas or spaces in the text box"
    intro = "Join the Nasdaq Community today and get free, instant access to portfolios, stock ratings, real-time alerts, and more!"
    processed_results = []
    for a in articles: 
        ad_string_pos = a.find(ad_string)
        intro_pos = a.find(intro)
        start_index = intro_pos + len(intro)
        end_index = ad_string_pos

        processed_results.append(a[start_index:end_index])
    
    return processed_results

In [ ]:
def get_all_articles(tickers, num_pages_to_read):
    all_articles = []
    for ticker in tickers:
        raw_articles = scrape_all_articles(ticker, num_pages_to_read)
        processed_articles = get_processed_articles(raw_articles)
        all_articles += processed_articles 
        print("Finished processing articles for: ", ticker)
    return all_articles


In [ ]:
def get_company_set(article, tickers):
    words_found = set([])
    ticker_set = set(tickers)
    article = article.split()
    for word in article:
        if word in ticker_set: 
            words_found.add(word)
    return words_found 
        
    

In [ ]:
def update_graph(edge_graph, company_set):
    pairs = itertools.combinations(company_set, 2)

    for pair in pairs: 
        if pair in edge_graph or (pair[1], pair[0]) in edge_graph:
            processed_pair = pair if pair in edge_graph else (pair[1], pair[0])
            edge_graph[processed_pair] += 1

In [ ]:
def read_articles_from_disk(data_directories):
    counter = 0
    articles = []
    for directory in data_directories: 
        file_list = os.listdir(directory)
        files_to_read = [os.path.join(directory, file) for file in file_list]
        for file in files_to_read: 
            with open(file, 'r') as content_file:
                content = content_file.read()
                articles.append(content)
    return articles
    

In [ ]:
directory_names = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DOW', 'GOOGL', 'GS', 'HD', 'IBM', 'KO', 'MMM', 'MSFT', 'NFLX', 'XOM']
root_data_dir = 'article_data'
data_directories = [os.path.join(root_data_dir, d) for d in directory_names]

all_articles = set(read_articles_from_disk(data_directories))

In [ ]:
s_p_500 = [s for s in pd.read_csv('constituents.csv', header=0)['Symbol']]
s_p_500.remove('A')
s_p_500.remove('GOOG')
s_p_500.remove('T')

In [ ]:
import itertools 
import random 
edges = itertools.combinations(s_p_500, 2)
edge_graph = {e:0 for e in edges}

for article in all_articles:
    company_set = get_company_set(article, s_p_500)
    update_graph(edge_graph, company_set)

In [ ]:
results = sorted([(v, k) for k, v in edge_graph.items()], reverse=True)
results

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.cubical_graph()
for edge in edge_graph:
    node1 = edge[0]
    node2 = edge[1]
    
    if node1 not in G.nodes():
        G.add_node(node1)
    
    if node2 not in G.nodes():
        G.add_node(node2)
for edge, e_weight in edge_graph.items():
    print(edge)
    node1 = edge[0]
    node2 = edge[1]
    
    G.add_edge(node1, node2, weight=e_weight)
    
nx.draw(G)
plt.show()

In [ ]:

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

comp_dict = {s:0 for s in s_p_500}

article_num = 0
for article in all_articles:
    doc = nlp(article)
    results = [(X.text, X.label_) for X in doc.ents]
    seen_set = set([])
    for r in results: 
        if r[1] == 'ORG':
            seen_set.add(r[0])
    print(seen_set)
    print(comp_dict['AAPL'])
    article_num += 1
    print("Done with: ", article_num)
comp_dict

In [ ]:
'''
tickers = ['IBM']
num_pages_to_read = 120
all_articles = get_all_articles(tickers, num_pages_to_read)
'''

In [ ]:
'''
ticker = tickers[0]
save_articles_to_files(all_articles, ticker)
'''

In [ ]:
'''
ticker_dictionary = {
    'GOOGL':set(['Google', 'Alphabet', 'GOOGL', 'googl', 'google']),
    'NFLX':set(['Netflix', 'NFLX', 'netflix', 'nflx']), 
    'MSFT':set(['MSFT', 'Microsoft', 'microsoft', 'MICROSOFT', 'msft']), 
    'AMZN':set(['AMZN', 'Amazon', 'amazon', 'amzn']), 
    'TSLA':set(['TSLA', 'TESLA', 'Tesla', 'tesla', 'tsla'])
}


dow_info = pd.read_html('https://finance.yahoo.com/quote/%5EDJI/components?p=%5EDJI')[0]['Symbol']
dow_tickers = [t for t in dow_info]
ticker_dictionary = {t:set([t]) for t in dow_tickers}

name_to_ticker = {}
for k, v, in ticker_dictionary.items():
    for name in v: 
        name_to_ticker[name] = k 
'''

In [ ]:
'''
import itertools 
edges = itertools.combinations(ticker_dictionary.keys(), 2)
edge_graph = {e:0 for e in edges}
num_pages_to_read = 100

all_articles = get_all_articles(ticker_dictionary.keys(), num_pages_to_read)

article_num = 0
for article in all_articles: 
    article_num += 1
    company_set = get_company_set(article, ticker_dictionary, name_to_ticker)
    update_graph(edge_graph, company_set)

    
print(edge_graph)
'''

In [ ]:
'''
results = [(v, k) for k, v in edge_graph.items()]
results = sorted(results, reverse=True)
print(results)
'''